In [139]:
import bs4
import requests

In [140]:
headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/69.0'})

## Binance

In [141]:

url = "https://api.binance.com/api/v1/ticker/24hr"
data = requests.get(url).json()

binance_dict = {}
for currency in data:
        name = currency['symbol']
        price = currency['lastPrice']
        volume = currency['volume']
        binance_dict[name] = [price, volume]    # binance_dict[name] = {цена, объем} 

## Okex

In [142]:
url = "https://okex.com/api/spot/v3/instruments/ticker/"
response = requests.get(url)
data = response.json()
okex_dict = {}
for currency in data:
    name = currency['instrument_id'].replace('-', '')  # убираем '-'
    price = currency['ask']
    volume = currency['base_volume_24h']
    okex_dict[name] = [price, volume]     # okex_dict[name] = {цена, объем}

In [143]:
ls1 = list(binance_dict.keys())   # списки названий инструментов каждой биржы
ls2 = list(okex_dict.keys())

In [144]:
all_names = set(ls1 + ls2)   # все неповторяющиеся названия интструментов

### DataFrame создаем

In [145]:
import pandas as pd
import numpy as np
unify_name = pd.Series([name for name in all_names])
binance_name = pd.Series([f'{price}$/{volume}' for name in all_names for price, volume in [binance_dict.get(name, ['-', '-'])]])
okex_name = pd.Series([f'{price}$/{volume}' for name in all_names for price, volume in [okex_dict.get(name, ['-', '-'])]])

### Сливаем все Серии вместе
Data = pd.DataFrame([unify_name, binance_name, okex_name]).T
Data.columns = ['unify_name', 'binance_name', 'okex_name']

In [146]:
Data.sort_values(by='unify_name', ignore_index=True)  ### сортировка по Unify_name

,unify_name,binance_name,okex_name
0,1INCHBTC,0.00005775$/1684890.90000000,-$/-
1,1INCHBUSD,2.18750000$/1479828.61000000,-$/-
2,1INCHDOWNUSDT,0.31430000$/7679457.31000000,-$/-
3,1INCHETH,-$/-,0.000946$/27307.11051512
4,1INCHUPUSDT,0.03322200$/113622142.05000000,-$/-
...,...,...,...
1771,ZRXBTC,0.00001826$/1496458.00000000,0.0000183$/1615263.488
1772,ZRXBUSD,0.69190000$/561186.97000000,-$/-
1773,ZRXETH,0.00029970$/117611.00000000,0.0003$/254783.466
1774,ZRXUSDT,0.69250000$/6435486.51000000,0.693$/5101895.603918
